In [2]:
import pyspark
from pyspark.sql.functions import *
sc = pyspark.SparkContext('local[*]')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-11-22T23:05:44,640 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-11-22T23:05:45,996 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.sql.functions import *

In [4]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [8]:
# 1
cleaned_olist_customers_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_geolocation_dataset.csv", header=True, inferSchema=True)
# 2
cleaned_olist_geolocation_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_geolocation_dataset.csv", header=True, inferSchema=True)
# 3
cleaned_olist_order_items_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_items_dataset.csv", header=True, inferSchema=True)
# 4
cleaned_olist_order_payments_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_payments_dataset.csv", header=True, inferSchema=True)
# 5
cleaned_olist_order_reviews_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_reviews_dataset.csv", header=True, inferSchema=True)
# 6
cleaned_olist_orders_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_orders_dataset.csv", header=True, inferSchema=True)
# 7
cleaned_olist_products_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_products_dataset.csv", header=True, inferSchema=True)
# 8
cleaned_olist_sellers_dataset = spark.read.csv("/obligatorio/data/cleaned/olist_sellers_dataset.csv", header=True, inferSchema=True)
# 9
cleaned_product_category_name_translation_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_product_category_name_translation.csv", header=True, inferSchema=True)

Es también conocido como diagrama estrella o diagrama de copo de nieve. Tenemos dos tipos de tablas, las de hechos, y las de dimensiones.

- Tablas de hechos: Contienen las medidas o los valores numéricos que se quieren analizar. Por lo general, los datos numéricos son agregados y analizados en diferentes niveles de granularidad, como por ejemplo, por día, semana, mes o trimestre. Estas tablas suelen tener muchas filas y relativamente pocas columnas, con cada fila representando una transacción u otra actividad registrada en el sistema.

- Tablas de dimensiones: Contienen información descriptiva que se utiliza para filtrar o segmentar los datos de la tabla de hechos. Estas tablas contienen atributos que describen las características de las transacciones, como la fecha, el lugar, el producto, el cliente, entre otros. Las tablas de dimensiones suelen tener menos filas y más columnas que las tablas de hechos.

Las tablas de hechos y las tablas de dimensiones están relacionadas entre sí a través de claves. Cada fila de la tabla de hechos se relaciona con una o más filas de la tabla de dimensiones a través de claves. Esta relación permite combinar los datos de las tablas de hechos y de dimensiones para realizar análisis detallados.

Damos forma a un esquema de estrella. Nuestras tablas de hechos van a ser el libro que es prestado y la ubicación del Libro. Nuestras dimensiones van a ser una de libros, una de librerías, otra de quienes piden prestados los libros y otra de publicadores.

Usamos claves surrogadas porque no tenemos control sobre los identificadores primarios de la fuente original. Así nos protegemos ante eventuales cambios.

Las tablas de dimensiones están hechas de tipo 1. Si queremos pasar a tipo 2 debemos agregar una columna flag que indique que estado de la fila es, o usar columnas con fechas para guardar el período de tiempo en el que estuvo el registro en ese estado, antes de pasar al siguiente.

Preguntas:

- ¿Conjunto de productos que más se venden juntos?
- Top 5 productos más vendidos por ciudad.
- ¿Qué días del mes se vende más?
- ¿Qué proporción de pedidos recibe cada calificación en las reseñas (de 1 a 5)? 
- ¿Cuál es el tiempo promedio entre la aprobación de un pedido y la entrega al cliente?

In [9]:
########################################################################################################################################

Database Schema

/bin/bash: -c: line 0: syntax error near unexpected token `attachment:obligatorio/database_schema.png'
/bin/bash: -c: line 0: `[obligatorio/database_schema.png](attachment:obligatorio/database_schema.png)'
